In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from tqdm import tqdm

from sklearn.metrics import ndcg_score

In [ ]:
import sys
sys.path.append('../src')

import config as cfg

In [ ]:
df_train = pd.read_csv(f'{cfg.DATASET_PATH}\\train.csv')
#df_test = pd.read_csv(f'{cfg.DATASET_PATH}\\test.csv')

df_songs = pd.read_csv(f'{cfg.DATASET_PATH}\\songs.csv')
df_members = pd.read_csv(f'{cfg.DATASET_PATH}\\members.csv')

In [ ]:
ALL_USERS = df_members['msno'].unique().tolist()
ALL_ITEMS = df_songs['song_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

df_train['user_id'] = df_train['msno'].map(user_map)
df_train['item_id'] = df_train['song_id'].map(item_map)

In [ ]:
del df_members, df_songs

In [ ]:
def calculate_top_musics(df: DataFrame, column_name: str = 'item_id') -> dict:
    music_metric = df.groupby(column_name).size().sort_values(ascending=False)
    music_top = music_metric.to_dict()
    return music_top

In [ ]:
top = calculate_top_musics(df_train)
top20_songs =[idx for idx, _ in top.items()][:20]


In [ ]:
average_ndcg_scores = []

# Чтобы лишний раз не считать оценку для конкретной песни, сразу выдадим им очки. Таким образом повышаем эффективность кода
top20_indices = {song: 20 - index for index, song in enumerate(top20_songs)}

for user_id in tqdm(df_train['user_id'].unique()):
    user_df = df_train[df_train['user_id'] == user_id]

    user_items = user_df['item_id'].tolist()
    true_relevance = user_df['target'].tolist()
    if len(true_relevance) > 1:
        # Оценки на основе позиций в ТОП20
        scores = np.zeros(len(user_items))
        for i, item in enumerate(user_items):
            scores[i] = top20_indices.get(item, 0)

        if np.sum(scores) > 0:  # Проверяем, есть ли среди оценок ненулевые значения
            ndcg_value = ndcg_score([true_relevance], [scores], k=20)
            average_ndcg_scores.append(ndcg_value)

average_ndcg = np.mean(average_ndcg_scores) if average_ndcg_scores else 0
print('Средний NDCG@20:', average_ndcg)

In [ ]:
#Средний NDCG@20: 0.6236679828362506